In [ ]:
import numpy as np
import pandas as pd
import cupy as cp
from typing import List, Tuple, Optional, Dict
import os
import matplotlib.pyplot as plt
import japanize_matplotlib
from scipy import signal, integrate
import cv2
from scipy.stats import entropy
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import UnivariateSpline

from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2lab
from scipy.spatial import ConvexHull

import pyVHR as vhr
from pyVHR.extraction.sig_processing import SignalProcessing
from pyVHR.plot.visualize import *
from pyVHR.BVP import *
vhr.plot.VisualizeParams.renderer = 'notebook'

import mediapipe as mp

In [ ]:
# 入力とする動画と動画のファイル名を取得
root_dir = "experimentData\\"
data_dirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
movie_paths = []
movie_names = []
true_value_csv_array = []
true_value_rri_csv_array = []
print("動画ディレクトリ:", data_dirs)

for i in range(len(data_dirs)):
    data_dir = data_dirs[i]

    # 動画ファイルのパスを取得
    movie_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.avi')]
    movie_paths.extend(movie_files)

    movie_name = os.path.basename(data_dir)
    movie_names.append(movie_name)

    # ppgファイルのパスを取得
    movie_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
    true_value_csv_array = [f.replace('.avi', '.csv') for f in movie_paths if f.endswith('.avi')]
    true_value_rri_csv_array.append(os.path.join(data_dir, 'RRI_Simple_' + movie_name + '.csv'))


f_1_ffi = 0.0399  # LFのはじめ
f_2 = 0.151  # LFの終わり、HFのはじめ
f_3 = 0.401  # HFの終わり

start_index = 6
# start_index = 17
# end_index = 18
end_index = len(movie_paths)
# start_index = end_index - 1  # 最後の動画のみを対象とする

data_dirs = data_dirs[start_index:end_index]
movie_paths = movie_paths[start_index:end_index]
movie_names = movie_names[start_index:end_index]
true_value_csv_array = true_value_csv_array[start_index:end_index]
true_value_rri_csv_array = true_value_rri_csv_array[start_index:end_index]

print(f"data_dirs: {data_dirs}")
print(f"movie_paths: {movie_paths}")
print(f"movie_names: {movie_names}")
print(f"true_value_csv_array: {true_value_csv_array}")
print(f"true_value_rri_csv_array: {true_value_rri_csv_array}")

In [ ]:
SAVE_DIR  = "POSAccuracyEvalu"
SPLIT_TIME = 90
WINDOW_SIZES = [2, 4, 6, 8, 10]
STRIDES = [2]

成果物
- 1フレームずつのRGBシグナル

## windowごとにRGB信号を分類

In [ ]:
def array_to_full_string(arr):
    """NumPy配列を省略なしの文字列に変換"""
    if isinstance(arr, str):
        return arr
    elif isinstance(arr, (np.ndarray, list)):
        # NumPy配列またはリストを完全な文字列に変換
        arr_np = np.array(arr)
        return np.array2string(arr_np, threshold=np.inf, max_line_width=np.inf, separator=' ')
    else:
        return str(arr)

In [ ]:
class StrideSegmentCalculator:
    def __init__(
        self,
        window_sizes: List[float] = [2, 3, 4, 5],
        strides: List[float] = [0.1, 0.5, 1, 1.5, 2],
    ):
        """
        Parameters:
        -----------
        window_sizes : List[float]
            窓幅（秒）のリスト
        strides : List[float]
            移動秒数のリスト
        """
        self.window_sizes = window_sizes
        self.strides = strides

    def calculate_overlap(self, window_size: float, stride: float) -> float:
        """
        窓幅と移動秒数からオーバーラップ率を計算

        Parameters:
        -----------
        window_size : float
            窓幅（秒）
        stride : float
            移動秒数（秒）

        Returns:
        --------
        float
            オーバーラップ率（%）
        """
        if stride >= window_size:
            return 0
        overlap = (window_size - stride) / window_size * 100
        return round(overlap, 2)

    def calculate_segments(
        self, window_size: float, stride: float, total_frames: int, fps: int
    ) -> List[Tuple[int, int]]:
        """
        フレーム数から解析区間を計算

        Parameters:
        -----------
        window_size : float
            窓幅（秒）
        stride : float
            移動秒数（秒）
        total_frames : int
            総フレーム数
        fps : int
            フレームレート

        Returns:
        --------
        List[Tuple[int, int]]
            各区間の(開始フレーム, 終了フレーム)のリスト
        """
        frames_per_window = round(window_size * fps)
        frames_per_stride = round(stride * fps)

        segments = []
        start_frame = 0

        while start_frame + frames_per_window <= total_frames:
            segments.append((start_frame, start_frame + frames_per_window))
            start_frame += frames_per_stride

        return segments

    def create_analysis_dataframe(self, total_frames: int, fps: int) -> pd.DataFrame:
        """
        全ての窓幅と移動秒数の組み合わせに対してDataFrameを生成

        Parameters:
        -----------
        total_frames : int
            総フレーム数
        fps : int
            フレームレート

        Returns:
        --------
        pd.DataFrame
            各条件でのセグメント情報を含むDataFrame
            columns: window_size, stride, overlap, segment_number, frame_start, frame_end
        """
        data_dict = {
            "window_size": [],
            "stride": [],
            "overlap": [],
            "segment_number": [],
            "frame_start": [],
            "frame_end": [],
        }

        for window_size in self.window_sizes:
            for stride in self.strides:
                overlap = self.calculate_overlap(window_size, stride)
                segments = self.calculate_segments(
                    window_size, stride, total_frames, fps
                )

                for i, (start_frame, end_frame) in enumerate(segments):
                    data_dict["window_size"].append(window_size)
                    data_dict["stride"].append(stride)
                    data_dict["overlap"].append(overlap)
                    data_dict["segment_number"].append(i)
                    data_dict["frame_start"].append(start_frame)
                    data_dict["frame_end"].append(end_frame)

        return pd.DataFrame(data_dict)


class PulseAnalysisDataStrides:
    def __init__(self, window_sizes, strides):
        # 窓枠とストライドの値を定義
        self.window_sizes = window_sizes
        self.strides = strides

        # accuracyのみを格納するDataFrameを初期化
        self.results = pd.DataFrame(
            index=pd.Index(self.window_sizes, name="window_size"),
            columns=pd.Index(self.strides, name="strides"),
        )

    def add_accuracy(self, window_size: float, strides: int, accuracy: float):
        """
        精度データを追加する

        Parameters:
        -----------
        window_size : float
            窓幅（秒）
        strides : int
            ストライド（s）
        accuracy : float
            精度値
        """
        self.results.loc[window_size, strides] = accuracy

    def _create_heatmap_dataframe(self):
        """
        ヒートマップ用のDataFrameを作成する内部メソッド

        Returns:
        --------
        pd.DataFrame
            ヒートマップ用に整形されたDataFrame
        """
        data = {"stride": self.strides}
        for window_size in self.window_sizes:
            data[window_size] = [
                self.results.loc[window_size, stride] for stride in self.strides
            ]
        df = pd.DataFrame(data).set_index("stride").T
        return df

    def save_heatmap(
        self,
        title: str,
        save_path: str,
        figsize: tuple = (10, 8),
        cmap: str = "YlGnBu",
        colorbar_label: str = "MAE",
    ):
        """
        ヒートマップを作成して保存する

        Parameters:
        -----------
        title : str
            プロットのタイトル
        save_path : str
            保存先のパス
        figsize : tuple, optional
            図のサイズ (default: (10, 8))
        cmap : str, optional
            カラーマップ (default: 'YlGnBu')
        colorbar_label : str, optional
            カラーバーのラベル (default: 'MAE')
        """
        df = self._create_heatmap_dataframe()

        # ヒートマップを作成
        plt.figure(figsize=figsize)
        sns.heatmap(
            df, annot=True, fmt=".4f", cmap=cmap, cbar_kws={"label": colorbar_label}
        )
        plt.title(f"{title}")
        plt.xlabel("Stride [s]")
        plt.ylabel("Window Size [s]")
        plt.tight_layout()
        plt.savefig(save_path, dpi=300)
        plt.close()

    def save_heatmap_std(self, title: str, save_path: str, figsize: tuple = (10, 8)):
        """
        標準偏差のヒートマップを作成して保存する

        Parameters:
        -----------
        title : str
            プロットのタイトル
        save_path : str
            保存先のパス
        figsize : tuple, optional
            図のサイズ (default: (10, 8))
        """
        self.save_heatmap(
            title,
            save_path,
            figsize,
            cmap="Reds",
            colorbar_label="Standard Deviation",
        )

In [ ]:
all_results_list = []
for i in range(len(movie_paths)):
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    print(f'Processing movie: {inputMoviePath}')

    # 動画のfpsを取得
    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    samplingRate = fps
    cap.release()
    
    # CSVファイルの読み込み
    ecg_csv_path = os.path.join(rootDir, dataName + '.csv')
    ecg_df = pd.read_csv(ecg_csv_path)
    
    ecg_RRI_csv_path = os.path.join(rootDir, f'RRI_Simple_{movie_names[i]}.csv')
    ecg_RRI_df = pd.read_csv(ecg_RRI_csv_path)
    
    stride_segment_calculator = StrideSegmentCalculator(window_sizes=WINDOW_SIZES, strides=STRIDES)
    analysis_df = stride_segment_calculator.create_analysis_dataframe(total_frames, fps)

    # RGB信号の読み込み
    save_roi_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(save_roi_dir, exist_ok=True)
    signals_csv_path = os.path.join(rootDir, f'extracted_signals.csv')
    signals_df = pd.read_csv(signals_csv_path)

    # 結果を格納するリスト
    all_window_results = []
    for idx, row in analysis_df.iterrows():
        window_size = row['window_size']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        stride = row['stride']

        # 窓の時間範囲を計算
        window_start_time = frame_start / fps
        window_end_time = frame_end / fps

        if(window_end_time > total_frames / fps):
            continue

        print(f"\nウィンドウ {idx}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {window_start_time:.2f}-{window_end_time:.2f}s")
            
        # 該当する窓の時間範囲内の真値RRIデータを抽出
        ecg_RRI_mask = (ecg_RRI_df['time'] >= window_start_time) & \
                (ecg_RRI_df['time'] < window_end_time)
        ecg_bpm_in_window = ecg_RRI_df[ecg_RRI_mask]['BPM'].values
        ecg_bpm_in_window_mean = np.mean(ecg_bpm_in_window) if len(ecg_bpm_in_window) > 0 else np.nan

        print(ecg_bpm_in_window)

        # 該当する窓の時間範囲内のRGB信号を抽出し、窓内でBVP算出
        bvp_mask = (signals_df['timestamp'] >= window_start_time) & (signals_df['timestamp'] < window_end_time)
        r_signal_in_window = signals_df[bvp_mask]['r_mean'].values
        g_signal_in_window = signals_df[bvp_mask]['g_mean'].values
        b_signal_in_window = signals_df[bvp_mask]['b_mean'].values


        # 窓情報を保存
        window_info = {
            'window_index': idx,
            'window_size': window_size,
            'stride': stride,
            'frame_start': frame_start,
            'frame_end': frame_end,
            'window_start_time': window_start_time,
            'window_end_time': window_end_time,
            'r_signal_in_window': array_to_full_string(r_signal_in_window),
            'g_signal_in_window': array_to_full_string(g_signal_in_window),
            'b_signal_in_window': array_to_full_string(b_signal_in_window),
            'ecg_bpm_in_window': array_to_full_string(ecg_bpm_in_window),
            'ecg_bpm_in_window_mean': ecg_bpm_in_window_mean,
        }
        all_window_results.append(window_info)

    # 全結果をDataFrameに変換して保存
    results_df = pd.DataFrame(all_window_results)
    results_csv_path = os.path.join(save_roi_dir, f'window_signals_{dataName}.csv')
    results_df.to_csv(results_csv_path, index=False, encoding='utf-8-sig')
    print(f"\n結果をCSVに保存: {results_csv_path}")

## 窓ごとにBVP解析
初期か処理のセクション2で同じことやってる

In [ ]:
def analyze_window_fft(values, fps):
    """
    時系列データの最大周波数とスペクトル情報を計算する関数

    Parameters:
    -----------
    values : array-like
        分析対象の時系列データ
    fps : int
        サンプリング周波数（1秒あたりのフレーム数）

    Returns:
    --------
    dict
        以下のキーを含む辞書：
        - 'max_freq': 検出された最大周波数
        - 'max_amplitude': 最大周波数のときの振幅
        - 'frequencies': 周波数配列（正の周波数のみ）
        - 'amplitudes': 振幅配列（正の周波数のみ）
        - 'power_spectrum': パワースペクトル
        - 'dominant_freqs': 上位5つの卓越周波数とその振幅
        - 'spectral_centroid': スペクトル重心
        - 'spectral_bandwidth': スペクトル帯域幅
        - 'total_power': 全体のパワー
    """
    # データをnumpy配列に変換
    values = np.array(values)

    # ゼロパディングで分解能を向上（窓長の8倍）
    n_pad = len(values) * 8

    # ハミング窓を適用（オプション：コメントアウトされている）
    # window = np.hamming(len(values))
    # windowed_data = values * window

    # FFTを実行（ゼロパディング適用）
    fft_result = np.fft.fft(values, n=n_pad)
    fft_freq = np.fft.fftfreq(n_pad, 1 / fps)

    # 正の周波数のみを取得
    positive_freq_idx = fft_freq > 0
    positive_fft = np.abs(fft_result[positive_freq_idx])
    positive_freq = fft_freq[positive_freq_idx]
    
    # パワースペクトルを計算
    power_spectrum = positive_fft ** 2

    # 最大周波数の検出と補間
    max_idx = np.argmax(positive_fft)
    max_amplitude = positive_fft[max_idx]
    
    if 0 < max_idx < len(positive_fft) - 1:
        # 3点を使用した放物線補間
        alpha = positive_fft[max_idx - 1]
        beta = positive_fft[max_idx]
        gamma = positive_fft[max_idx + 1]
        peak_pos = 0.5 * (alpha - gamma) / (alpha - 2 * beta + gamma)

        # 補間された周波数と振幅
        freq_resolution = fps / n_pad
        max_freq = positive_freq[max_idx] + peak_pos * freq_resolution
        
        # 補間された振幅（放物線の頂点）
        max_amplitude = beta - 0.25 * (alpha - gamma) * peak_pos
    else:
        max_freq = positive_freq[max_idx]
    
    # 上位5つの卓越周波数を検出
    top_indices = np.argsort(positive_fft)[-5:][::-1]
    dominant_freqs = [(positive_freq[idx], positive_fft[idx]) for idx in top_indices]
    
    # スペクトル特徴量の計算
    # スペクトル重心（周波数の重み付き平均）
    spectral_centroid = np.sum(positive_freq * positive_fft) / np.sum(positive_fft)
    
    # スペクトル帯域幅（重心からの重み付き分散）
    spectral_bandwidth = np.sqrt(
        np.sum(((positive_freq - spectral_centroid) ** 2) * positive_fft) / np.sum(positive_fft)
    )
    
    # 全体のパワー
    total_power = np.sum(power_spectrum)
    
    # 結果を辞書にまとめる
    result = {
        'max_freq': max_freq,
        'max_amplitude': max_amplitude,
        'frequencies': positive_freq,
        'amplitudes': positive_fft,
        'power_spectrum': power_spectrum,
        'dominant_freqs': dominant_freqs,
        'spectral_centroid': spectral_centroid,
        'spectral_bandwidth': spectral_bandwidth,
        'total_power': total_power,
        'freq_resolution': fps / n_pad,  # 周波数分解能
        'nyquist_freq': fps / 2  # ナイキスト周波数
    }
    
    return result

In [ ]:
def extract_BVPsignal(r_signals, g_signals, b_signals,  fps, deviceType, bvpMethod, bvpMethodName, method_params=None):
    """
    RGB信号からBVP信号を抽出する関数
    """
    rgb_signal = np.array([[r_signals, g_signals, b_signals]], dtype=np.float32)
    print(f"\nRGB信号の形状: {rgb_signal.shape}")
    
    signal_length = rgb_signal.shape[2]
    min_required_length = 50
    
    if signal_length < min_required_length:
        print(f"警告: 信号長が短すぎます ({signal_length} < {min_required_length})。処理をスキップします。")
        return None, None
    
    filtered_signal = [rgb_signal]
    
    # デフォルトのメソッドパラメータ
    if method_params is None:
        method_params = {}
    
    # メソッド別パラメータ設定
    if bvpMethodName in ["cupy_POS", "cpu_POS"]:
        method_params['fps'] = fps
    elif bvpMethodName in ["cpu_ICA", "cpu_PCA"]:
        method_params['component'] = 'all_comp'
    
    print(f"\nBVP抽出開始 (メソッド: {bvpMethodName})")
    print(f"パラメータ: {method_params}")
    
    # BVP信号抽出
    if method_params:
        bvp_signal = vhr.BVP.RGB_sig_to_BVP(
            filtered_signal,
            fps,
            device_type=deviceType,
            method=bvpMethod,
            params=method_params
        )
    else:
        bvp_signal = vhr.BVP.RGB_sig_to_BVP(
            filtered_signal,
            fps,
            device_type=deviceType,
            method=bvpMethod
        )
    
    # 生のBVP信号を保存
    raw_bvp_signal = bvp_signal[0].copy() if len(bvp_signal) > 0 else None
    
    # 後処理フィルタリング
    bvp_signal = vhr.BVP.apply_filter(
        bvp_signal,
        vhr.BVP.BPfilter,
        params={'order': 6, 'minHz': 0.5, 'maxHz': 2.0, 'fps': fps}
    )
    
    bvp_signal = vhr.BVP.apply_filter(bvp_signal, vhr.BVP.zeromean)
    
    filtered_bvp_signal = bvp_signal[0] if len(bvp_signal) > 0 else None
    
    print(f"\nBVP信号抽出完了")
    return raw_bvp_signal, filtered_bvp_signal

In [ ]:
# メソッドの組み合わせを定義
methodCombinations = [
    # 拡張POS（蒸気対応）
    ['cuda', cupy_CHROM, "cupy_CHROM"],
    ['cpu', cpu_LGI, "cpu_LGI"],
    ['cpu', cpu_GREEN, "cpu_GREEN"],
    ['cpu', cpu_ICA, "cpu_ICA"],
    ['cuda', cupy_POS, "cupy_POS"]
]

for i in range(len(movie_paths)):
    print(f'Processing movie: {movie_paths[i]}')
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # CSVファイルの読み込み
    ecg_csv_path = os.path.join(rootDir, dataName + '.csv')
    ecg_df = pd.read_csv(ecg_csv_path)

    ecg_RRI_csv_path = os.path.join(rootDir, f'RRI_Simple_{dataName}.csv')
    ecg_RRI_df = pd.read_csv(ecg_RRI_csv_path)

    ecg_bpm_in_window = ecg_RRI_df['BPM']
    ecg_bpm_in_window_mean = ecg_RRI_df['BPM'].values.mean()

    # window_signalsの読み込み
    os.makedirs(os.path.join(rootDir, SAVE_DIR), exist_ok=True)
    window_signals_data_path = os.path.join(rootDir, SAVE_DIR, f'{"window_signals_" + dataName}.csv')
    window_signals_data_df = pd.read_csv(window_signals_data_path)
    print(f'Loaded window signals data from {window_signals_data_path}, {len(window_signals_data_df)} rows')

    stride_segment_calculator = StrideSegmentCalculator(window_sizes=WINDOW_SIZES, strides=STRIDES)
    analysis_df = stride_segment_calculator.create_analysis_dataframe(total_frames, fps)

    # 結果を格納するリスト
    all_window_results = []
    
    for idx, row in analysis_df.iterrows():
        window_size = row['window_size']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        stride = row['stride']

        window_start_time = frame_start / fps
        window_end_time = frame_end / fps

        if window_end_time > total_frames / fps:
            continue

        print(f"\nウィンドウ {idx}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {window_start_time:.2f}-{window_end_time:.2f}s")

        # 真値RRIデータを抽出
        ecg_RRI_mask = (ecg_RRI_df['time'] >= window_start_time) & \
                       (ecg_RRI_df['time'] < window_end_time)
        ecg_bpm_in_window = ecg_RRI_df[ecg_RRI_mask]['BPM'].values
        ecg_bpm_in_window_mean = np.mean(ecg_bpm_in_window) if len(ecg_bpm_in_window) > 0 else np.nan
        print(f'    True Value RRI count in window: {len(ecg_bpm_in_window)}, Mean BPM: {ecg_bpm_in_window_mean:.2f}')

        # RGB信号を抽出
        bvp_mask = window_signals_data_df['window_index'] == idx
        print(f'    BVP mask has {bvp_mask.sum()} matching rows')

        filtered_rows = window_signals_data_df[bvp_mask]

        r_signal_in_window = filtered_rows['r_signal_in_window'].values[0]
        g_signal_in_window = filtered_rows['g_signal_in_window'].values[0]
        b_signal_in_window = filtered_rows['b_signal_in_window'].values[0]

        # 文字列をNumPy配列に変換
        r_signal_in_window = np.fromstring(r_signal_in_window[1:-1], sep=' ')
        g_signal_in_window = np.fromstring(g_signal_in_window[1:-1], sep=' ')
        b_signal_in_window = np.fromstring(b_signal_in_window[1:-1], sep=' ')
        
        if len(r_signal_in_window) == 0 or len(g_signal_in_window) == 0 or len(b_signal_in_window) == 0:
            raise ValueError("RGB信号が窓内に存在しません")

        # BVPメソッドの設定
        for methodCombination in methodCombinations:
            deviceType = methodCombination[0] 
            bvpMethod = methodCombination[1] 
            bvpMethodName = methodCombination[2]
            method_params = methodCombination[3].copy() if len(methodCombination) > 3 else {}


            print(f"\n{'='*60}")
            print(f"実行メソッド: {bvpMethodName}")
            print(f"{'='*60}")

            # rgbからBVPを計算
            raw_bvp_signal_in_window, filtered_bvp_signal_in_window = extract_BVPsignal(
                r_signal_in_window,
                g_signal_in_window,
                b_signal_in_window,
                fps,
                deviceType,
                bvpMethod,
                bvpMethodName,
                method_params=method_params
            )
            
            # BVP信号の抽出に失敗した場合はスキップ
            if filtered_bvp_signal_in_window is None:
                print(f"ウィンドウ {idx} をスキップ: BVP信号の抽出に失敗")
                continue
            
            # FFT解析
            raw_bvp_signal_in_window = raw_bvp_signal_in_window.flatten() if raw_bvp_signal_in_window is not None else None
            filtered_bvp_signal_in_window = filtered_bvp_signal_in_window.flatten()
            fft_result_dic = analyze_window_fft(filtered_bvp_signal_in_window, fps)

            # MAEの計算
            rppg_bpm = fft_result_dic['max_freq'] * 60
            rppg_freq = fft_result_dic['frequencies']
            rppg_amplitude = fft_result_dic['amplitudes']
            rppg_pwd = fft_result_dic['power_spectrum']

            bpm_MAE = np.abs(ecg_bpm_in_window_mean - rppg_bpm) if not np.isnan(ecg_bpm_in_window_mean) else np.nan

            print(f"\n結果サマリー:")
            print(f"  ECG BPM: {ecg_bpm_in_window_mean:.2f}")
            print(f"  rPPG BPM: {rppg_bpm:.2f}")
            print(f"  MAE: {bpm_MAE:.2f}")

            # 窓情報を保存
            window_info = {
                'window_index': idx,
                'bvp_method': bvpMethodName,
                'window_size': window_size,
                'stride': stride,
                'frame_start': frame_start,
                'frame_end': frame_end,
                'window_start_time': window_start_time,
                'window_end_time': window_end_time,
                'r_signal_in_window': array_to_full_string(r_signal_in_window),
                'g_signal_in_window': array_to_full_string(g_signal_in_window),
                'b_signal_in_window': array_to_full_string(b_signal_in_window),
                'raw_bvp_in_window': raw_bvp_signal_in_window,
                'filtered_bvp_in_window': filtered_bvp_signal_in_window,
                'ecg_bpm_in_window': ecg_bpm_in_window,
                'ecg_bpm_mean': ecg_bpm_in_window_mean,
                'rppg_bpm': rppg_bpm,
                'bpm_MAE': bpm_MAE,
                'max_freq': fft_result_dic['max_freq'],
                'max_amplitude': fft_result_dic['max_amplitude'],
                'spectral_centroid': fft_result_dic['spectral_centroid'],
                'spectral_bandwidth': fft_result_dic['spectral_bandwidth'],
                'total_power': fft_result_dic['total_power']
            }
            all_window_results.append(window_info)
    
    # 全結果をDataFrameに変換して保存
    results_df = pd.DataFrame(all_window_results)
    results_save_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(results_save_dir, exist_ok=True)
    results_csv_path = os.path.join(results_save_dir, f'window_analysis_{dataName}.csv')
    results_df.to_csv(results_csv_path, index=False, encoding='utf-8-sig')
    print(f"\nFFT結果をCSVに保存: {results_csv_path}")  

In [ ]:
# 分割時間の設定(秒)

# 全ての動画データを処理
for i in range(len(movie_paths)):
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    print(f'\n{"="*60}')
    print(f'Processing movie: {dataName}')
    print(f'{"="*60}')
    
    # 前半と後半でデータを分割
    results_df = pd.read_csv(os.path.join(rootDir, SAVE_DIR, f'window_analysis_{dataName}.csv'))
    first_half_mask = results_df['window_end_time'] <= SPLIT_TIME
    second_half_mask = results_df['window_start_time'] >= SPLIT_TIME

    first_half_df = results_df[first_half_mask].copy()
    second_half_df = results_df[second_half_mask].copy()

    print(f'前半データ数: {len(first_half_df)}, 後半データ数: {len(second_half_df)}')
    
    # ヒートマップを作成する関数
    def create_mae_heatmap(df, title_suffix, save_suffix):
        """
        BPM MAEのヒートマップを作成
        
        Parameters:
        -----------
        df : pd.DataFrame
            window_analysisデータ
        title_suffix : str
            タイトルに追加する文字列
        save_suffix : str
            保存ファイル名に追加する文字列
        """
        # 手法のリストを取得
        methods = df['bvp_method'].unique()
        window_sizes = sorted(df['window_size'].unique())
        
        # ヒートマップ用のデータを準備
        heatmap_data = pd.DataFrame(index=window_sizes, columns=methods)
        
        for window_size in window_sizes:
            for method in methods:
                mask = (df['window_size'] == window_size) & (df['bvp_method'] == method)
                mae_values = df[mask]['bpm_MAE'].dropna()
                
                if len(mae_values) > 0:
                    # 平均MAEを計算
                    mean_mae = mae_values.mean()
                    heatmap_data.loc[window_size, method] = mean_mae
        
        # データ型を数値に変換
        heatmap_data = heatmap_data.astype(float)
        
        # ヒートマップを作成
        plt.figure(figsize=(12, 8))
        sns.heatmap(
            heatmap_data,
            annot=True,
            fmt='.4f',
            cmap='YlOrRd',
            cbar_kws={'label': 'Mean MAE (BPM)'},
            vmin=0,
            vmax=25
        )
        
        plt.title(f'MAE vs BVP Method - {dataName} ({title_suffix})', fontsize=14, pad=20)
        plt.xlabel('BVP Method', fontsize=12)
        plt.ylabel('Window Size [s]', fontsize=12)
        plt.tight_layout()
        
        # 保存
        save_dir = os.path.join(rootDir, SAVE_DIR)
        save_path = os.path.join(save_dir, f'heatmap_mae_{save_suffix}_{dataName}.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f'  ヒートマップを保存: {save_path}')
        
        return heatmap_data
    
    # 前半のヒートマップを作成
    if len(first_half_df) > 0:
        print('\n前半(ノイズなし)のヒートマップを作成中...')
        first_half_heatmap = create_mae_heatmap(
            first_half_df,
            'First Half (No Noise)',
            'first_half'
        )
    else:
        print('前半のデータが存在しません')
    
    # 後半のヒートマップを作成
    if len(second_half_df) > 0:
        print('\n後半(ノイズあり)のヒートマップを作成中...')
        second_half_heatmap = create_mae_heatmap(
            second_half_df,
            'Second Half (With Noise)',
            'second_half'
        )
    else:
        print('後半のデータが存在しません')
    
    # 統計情報を出力
    print(f'\n統計情報 - {dataName}')
    print('-' * 60)
    
    if len(first_half_df) > 0:
        print('前半(ノイズなし):')
        print(f'  平均MAE: {first_half_df["bpm_MAE"].mean():.4f}')
        print(f'  最小MAE: {first_half_df["bpm_MAE"].min():.4f}')
        print(f'  最大MAE: {first_half_df["bpm_MAE"].max():.4f}')
    
    if len(second_half_df) > 0:
        print('\n後半(ノイズあり):')
        print(f'  平均MAE: {second_half_df["bpm_MAE"].mean():.4f}')
        print(f'  最小MAE: {second_half_df["bpm_MAE"].min():.4f}')
        print(f'  最大MAE: {second_half_df["bpm_MAE"].max():.4f}')

print('\n全ての処理が完了しました!')  

### POSの射影行列を系統的に変えることで精度が向上するかをテスト

In [ ]:
PROJECTION_MATRICES = {
    'original': [[-2, 1, 1]],
    'modified': [[-1, 1, 1]],
    'variant1': [[-0.5, 1, 1]],  # 追加例
}

In [ ]:
methodCombinations = [
    ['cuda', cupy_POS, "cupy_POS"]
]

for i in range(len(movie_paths)):
    print(f'Processing movie: {movie_paths[i]}')
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    window_analysis_path = os.path.join(rootDir, SAVE_DIR, f'window_analysis_{dataName}.csv')
    window_analysis_df = pd.read_csv(window_analysis_path)

    # 手法がcupy_POSかつbpm_MAEが10以上のデータを抽出
    high_mae_df = window_analysis_df[window_analysis_df['bvp_method'] == 'cupy_POS']
    high_mae_df = high_mae_df[high_mae_df['bpm_MAE'] >= 10]
    print(f'\n高MAEデータ (MAE >= 10) - {dataName}, 件数: {len(high_mae_df)}')

    # 高MAEデータのbpm_MAEの窓長ごとの平均値を計算
    if len(high_mae_df) > 0:
        mean_high_mae = high_mae_df.groupby('window_size')['bpm_MAE'].mean()
        print(f'  高MAEデータの平均MAE:\n{mean_high_mae}')
    else:
        print('  高MAEデータが存在しません')

    # high_mae_dfの各行を処理
    for idx, row in high_mae_df.iterrows():
        window_index = row['window_index']
        window_size = row['window_size']
        stride = row['stride']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        window_start_time = row['window_start_time']
        window_end_time = row['window_end_time']

        print(f"\nウィンドウ {window_index}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {window_start_time:.2f}-{window_end_time:.2f}s, MAE: {row['bpm_MAE']:.2f}")

        # RGB信号を抽出
        r_signal_in_window = np.fromstring(row['r_signal_in_window'][1:-1], sep=' ')
        g_signal_in_window = np.fromstring(row['g_signal_in_window'][1:-1], sep=' ')
        b_signal_in_window = np.fromstring(row['b_signal_in_window'][1:-1], sep=' ')

        print(f'  R信号長: {len(r_signal_in_window)}, G信号長: {len(g_signal_in_window)}, B信号長: {len(b_signal_in_window)}')

        # rgbからBVPを計算
        rgb_signal = np.array([[r_signal_in_window, g_signal_in_window, b_signal_in_window]], dtype=np.float32)
        print(f"\nRGB信号の形状: {rgb_signal.shape}")

        signal_length = rgb_signal.shape[2]
        min_required_length = 50
            
        filtered_signal = [rgb_signal]
            
        # ============================================================================
        # 【重要】POS法を直接実装（調整可能）
        # ============================================================================
        import cupy as cp

        # CuPy配列に変換
        rgb_cupy = cp.asarray(rgb_signal)

        # POS法のパラメータ
        eps = 10**-9
        X = rgb_cupy
        fps_cupy = cp.float32(fps)
        e, c, f = X.shape  # e = #estimators, c = 3 rgb ch., f = #frames
        w = int(1.6 * fps_cupy)  # window length

        # 投影行列P
        P = cp.array([[0, 1, -1], [-2, 1, 1]])
        Q = cp.stack([P for _ in range(e)], axis=0)

        # 初期化
        H = cp.zeros((e, f))

        # 診断情報を保存するリスト
        alpha_list = []
        M_list = []
        S1_list = []
        S2_list = []
        alpha_S2_list = []
        window_indices = []

        # スライディングウィンドウループ
        for n in cp.arange(w, f):
            m = n - w + 1
            
            # 時間的正規化
            Cn = X[:, :, m:(n + 1)]
            M = 1.0 / (cp.mean(Cn, axis=2) + eps)
            M_expanded = cp.expand_dims(M, axis=2)
            Cn = cp.multiply(M_expanded, Cn)
            
            # Mの値を保存
            M_list.append(cp.asnumpy(M))
            
            # 投影
            S = cp.dot(Q, Cn)
            S = S[0, :, :, :]
            S = cp.swapaxes(S, 0, 1)
            
            # チューニング
            S1 = S[:, 0, :]
            S2 = S[:, 1, :]
            alpha = cp.std(S1, axis=1) / (eps + cp.std(S2, axis=1))
            
            # S1とS2を保存
            S1_list.append(cp.asnumpy(S1))
            S2_list.append(cp.asnumpy(S2))
            
            # alphaの値を保存
            alpha_list.append(cp.asnumpy(alpha))
            
            alpha_expanded = cp.expand_dims(alpha, axis=1)
            # alphaを0に固定
            # alpha_expanded = cp.zeros_like(alpha_expanded)
            alpha_S2 = alpha_expanded * S2
            
            # alpha*S2を保存
            alpha_S2_list.append(cp.asnumpy(alpha_S2))
            
            window_indices.append(int(n))
            
            Hn = cp.add(S1, alpha_S2)
            Hnm = Hn - cp.expand_dims(cp.mean(Hn, axis=1), axis=1)
            
            # オーバーラップ加算
            H[:, m:(n + 1)] = cp.add(H[:, m:(n + 1)], Hnm)

        # 診断情報を辞書にまとめる
        diagnostics_info = {
            'alpha_history': np.array(alpha_list),
            'M_history': np.array(M_list),
            'S1_history': np.array(S1_list),
            'S2_history': np.array(S2_list),
            'alpha_S2_history': np.array(alpha_S2_list),
            'window_indices': np.array(window_indices)
        }

        # NumPy配列に戻す
        bvp_cupy = H
        bvp_numpy = cp.asnumpy(bvp_cupy)

        raw_bvp_signal = [bvp_numpy]
        bvp_signal = [bvp_numpy.copy()]

        # 後処理フィルタリング
        bvp_signal = vhr.BVP.apply_filter(
            bvp_signal,
            vhr.BVP.BPfilter,
            params={'order': 6, 'minHz': 0.5, 'maxHz': 2.0, 'fps': fps}
        )

        bvp_signal = vhr.BVP.apply_filter(bvp_signal, vhr.BVP.zeromean)

        raw_bvp_signal_in_window = raw_bvp_signal[0] if len(bvp_signal) > 0 else None
        filtered_bvp_signal_in_window = bvp_signal[0] if len(bvp_signal) > 0 else None

        # FFT解析
        raw_bvp_signal_in_window = raw_bvp_signal_in_window.flatten() if raw_bvp_signal_in_window is not None else None
        filtered_bvp_signal_in_window = filtered_bvp_signal_in_window.flatten()
        fft_result_dic = analyze_window_fft(filtered_bvp_signal_in_window, fps)

        # MAEの計算
        rppg_bpm = fft_result_dic['max_freq'] * 60
        rppg_freq = fft_result_dic['frequencies']
        rppg_amplitude = fft_result_dic['amplitudes']
        rppg_pwd = fft_result_dic['power_spectrum']

        bpm_MAE = np.abs(ecg_bpm_in_window_mean - rppg_bpm) if not np.isnan(ecg_bpm_in_window_mean) else np.nan

        print(f"\n結果サマリー:")
        print(f"  ECG BPM: {ecg_bpm_in_window_mean:.2f}")
        print(f"  rPPG BPM: {rppg_bpm:.2f}")
        print(f"  MAE: {bpm_MAE:.2f}")

### POS拡張

In [ ]:
def redesign_projection_matrix(X_apparent, u_pbv):
    """
    適応的射影行列を再設計
    
    Args:
        X_apparent: (3,) 見かけの肌色調ベクトル
        u_pbv: (3,) 拍動ベクトル [0.33, 0.77, 0.53]
    
    Returns:
        P_adaptive: (2, 3) 再設計された射影行列
    """
    eps = 1e-9
    X_apparent = cp.asarray(X_apparent)
    
    # u_pbv から X_apparent 成分を除去（直交化）
    dot_product = cp.dot(u_pbv, X_apparent)
    u_proj = u_pbv - dot_product * X_apparent
    u_proj_norm = cp.linalg.norm(u_proj)
    
    # 安全性チェック: ほぼ平行な場合はフォールバック
    if u_proj_norm < 0.1:
        print(f"  [警告] u_pbv と X_apparent の角度が小さすぎる (norm={u_proj_norm:.4f})")
        print(f"  標準POSにフォールバック")
        # 標準の射影行列を返す
        P_standard = cp.array([[0, 1, -1], [-2, 1, 1]], dtype=cp.float32)
        return P_standard
    
    # 第1軸: 拍動方向（u_skinに直交）
    z1 = u_proj / u_proj_norm
    
    # 第2軸: z1 と X_apparent の両方に直交
    z2 = cp.cross(X_apparent, z1)
    z2_norm = cp.linalg.norm(z2)
    z2 = z2 / (z2_norm + eps)
    
    # 拍動方向チェック（z2 も正の拍動性を持つように）
    if cp.dot(z2, u_pbv) < 0:
        z2 = -z2
    
    # 射影行列を構築
    P_adaptive = cp.stack([z1, z2], axis=0)
    
    return P_adaptive

In [ ]:
import cupy as cp

# ============================================================================
# メイン処理部分（既存コードに組み込む）
# ============================================================================

# 結果を保存するリスト
all_results = []

for i in range(len(movie_paths)):
    print(f'Processing movie: {movie_paths[i]}')
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    window_analysis_path = os.path.join(rootDir, SAVE_DIR, f'window_analysis_{dataName}.csv')
    window_analysis_df = pd.read_csv(window_analysis_path)

    # cupy_POSかつMAE >= 10のデータを抽出
    high_mae_df = window_analysis_df[window_analysis_df['bvp_method'] == 'cupy_POS']
    high_mae_df = high_mae_df[high_mae_df['bpm_MAE'] >= 10]
    print(f'\n高MAEデータ (MAE >= 10) - {dataName}, 件数: {len(high_mae_df)}')

    if len(high_mae_df) == 0:
        print('  高MAEデータが存在しません')
        continue

    all_results = []
    # 各ウィンドウを処理
    for idx, row in high_mae_df.iterrows():
        window_index = row['window_index']
        window_size = row['window_size']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        original_mae = row['bpm_MAE']

        # print(f"\n{'='*80}")
        # print(f"ウィンドウ {window_index}: フレーム {frame_start}-{frame_end}, 元のMAE: {original_mae:.2f}")
        # print(f"{'='*80}")

        # RGB信号を抽出
        r_signal_in_window = np.fromstring(row['r_signal_in_window'][1:-1], sep=' ')
        g_signal_in_window = np.fromstring(row['g_signal_in_window'][1:-1], sep=' ')
        b_signal_in_window = np.fromstring(row['b_signal_in_window'][1:-1], sep=' ')

        rgb_signal = np.array([[r_signal_in_window, g_signal_in_window, b_signal_in_window]], dtype=np.float32)
        
        # ECG BPM（真値）
        ecg_bpm = row['ecg_bpm_in_window']
        ecg_bpm = np.fromstring(ecg_bpm[1:-1], sep=' ') if isinstance(ecg_bpm, str) else np.array([])
        ecg_bpm_in_window_mean = ecg_bpm.mean() if ecg_bpm is not None else None

        # 各射影行列で処理
        # CuPy配列に変換
        rgb_cupy = cp.asarray(rgb_signal)

        # POS法のパラメータ
        eps = 10**-9
        X = rgb_cupy
        fps_cupy = cp.float32(fps)
        e, c, f = X.shape  # e = #estimators, c = 3 rgb ch., f = #frames
        w = int(1.6 * fps_cupy)  # window length (論文では32フレーム @ 20fps)

        # 固定の拍動ベクトル（論文の式30）
        u_pbv = cp.array([0.33, 0.77, 0.53], dtype=cp.float32)

        # 肌色調 + 蒸気ベクトル
        r_signal_in_window_square = r_signal_in_window ** 2
        g_signal_in_window_square = g_signal_in_window ** 2
        b_signal_in_window_square = b_signal_in_window ** 2

        normalized_r_signal_in_window = r_signal_in_window / np.sqrt(r_signal_in_window_square + g_signal_in_window_square + b_signal_in_window_square)
        normalized_g_signal_in_window = g_signal_in_window / np.sqrt(r_signal_in_window_square + g_signal_in_window_square + b_signal_in_window_square)
        normalized_b_signal_in_window = b_signal_in_window / np.sqrt(r_signal_in_window_square + g_signal_in_window_square + b_signal_in_window_square)

        skin_vector_array = np.array([normalized_r_signal_in_window, normalized_g_signal_in_window, normalized_b_signal_in_window])
        skin_vector = cp.mean(cp.asarray(skin_vector_array), axis=1)

        # 初期化
        H = cp.zeros((e, f))

        X_apparent_current = None
        P_current = None

        # POS法のパラメータ
        eps = 10**-9
        X = rgb_cupy
        fps_cupy = cp.float32(fps)
        e, c, f = X.shape  # e = #estimators, c = 3 rgb ch., f = #frames
        w = int(1.6 * fps_cupy)  # window length

        # 投影行列P
        # P = redesign_projection_matrix(skin_vector, u_pbv)
        P = cp.array([[0, 1, -1], [-2, 1, 1]])
        Q = cp.stack([P for _ in range(e)], axis=0)

        alpha_list = []

        # 初期化
        H = cp.zeros((e, f))

        # スライディングウィンドウループ
        for n in cp.arange(w, f):
            m = n - w + 1
            
            # 時間的正規化
            Cn = X[:, :, m:(n + 1)]
            M = 1.0 / (cp.mean(Cn, axis=2) + eps)
            M_expanded = cp.expand_dims(M, axis=2)
            Cn = cp.multiply(M_expanded, Cn)
            
            # 投影
            S = cp.dot(Q, Cn)
            S = S[0, :, :, :]
            S = cp.swapaxes(S, 0, 1)
            
            # チューニング
            S1 = S[:, 0, :]
            S2 = S[:, 1, :]

            # S2から定常成分である1を引く
            S2 = S2 - 1.0
            
            alpha = cp.std(S1, axis=1) / (eps + cp.std(S2, axis=1))
            alpha_list.append(cp.asnumpy(alpha))
            
            alpha_expanded = cp.expand_dims(alpha, axis=1)
            
            # alphaを0に固定
            # alpha_expanded = cp.zeros_like(alpha_expanded)
            alpha_S2 = alpha_expanded * S2
            
            Hn = cp.add(S1, alpha_S2)
            Hnm = Hn - cp.expand_dims(cp.mean(Hn, axis=1), axis=1)
            
            # オーバーラップ加算
            H[:, m:(n + 1)] = cp.add(H[:, m:(n + 1)], Hnm)

            # S1, S2をプロット
            # 1段目がS1、2段目がS2
            # S1, S2をプロット
            # fig, axes = plt.subplots(2, 1, figsize=(10, 6))
            
            # # S1信号のプロット
            # S1_np = cp.asnumpy(S1[0, :])
            # axes[0].plot(S1_np, color='blue', label='S1 Signal', linewidth=1.5)
            # axes[0].set_title(f'Window {window_index} - Frame {int(m)} to {int(n)} - S1 Signal')
            # axes[0].set_xlabel('Frame')
            # axes[0].set_ylabel('Amplitude')
            # axes[0].legend()
            # axes[0].grid(True, alpha=0.3)
            
            # # S2信号のプロット
            # S2_np = cp.asnumpy(S2[0, :])
            # axes[1].plot(S2_np, color='orange', label='S2 Signal', linewidth=1.5)
            # axes[1].set_title(f'Window {window_index} - Frame {int(m)} to {int(n)} - S2 Signal')
            # axes[1].set_xlabel('Frame')
            # axes[1].set_ylabel('Amplitude')
            # axes[1].legend()
            # axes[1].grid(True, alpha=0.3)
            
            # plt.tight_layout()
            # plot_save_dir = os.path.join(rootDir, SAVE_DIR, 'adaptive_removeDC_diagnostics_plots')
            # os.makedirs(plot_save_dir, exist_ok=True)
            # plot_filename = f'window_{window_index}_{int(n)}_S1_S2_signals.png'
            # plot_path = os.path.join(plot_save_dir, plot_filename)
            # plt.savefig(plot_path)
            # plt.close()

        # NumPy配列に戻す
        bvp_cupy = H
        bvp_numpy = cp.asnumpy(bvp_cupy)

        raw_bvp_signal = [bvp_numpy]
        bvp_signal = [bvp_numpy.copy()]

        # 後処理フィルタリング
        bvp_signal = vhr.BVP.apply_filter(
            bvp_signal,
            vhr.BVP.BPfilter,
            params={'order': 6, 'minHz': 0.5, 'maxHz': 2.0, 'fps': fps}
        )

        bvp_signal = vhr.BVP.apply_filter(bvp_signal, vhr.BVP.zeromean)

        raw_bvp_signal_in_window = raw_bvp_signal[0] if len(bvp_signal) > 0 else None
        filtered_bvp_signal_in_window = bvp_signal[0] if len(bvp_signal) > 0 else None

        # FFT解析
        raw_bvp_signal_in_window = raw_bvp_signal_in_window.flatten() if raw_bvp_signal_in_window is not None else None
        filtered_bvp_signal_in_window = filtered_bvp_signal_in_window.flatten()
        fft_result_dic = analyze_window_fft(filtered_bvp_signal_in_window, fps)

        # MAEの計算
        rppg_bpm = fft_result_dic['max_freq'] * 60
        rppg_freq = fft_result_dic['frequencies']
        rppg_amplitude = fft_result_dic['amplitudes']
        rppg_pwd = fft_result_dic['power_spectrum']

        bpm_MAE = np.abs(ecg_bpm_in_window_mean - rppg_bpm) if not np.isnan(ecg_bpm_in_window_mean) else np.nan

        # 結果をリストに追加
        # 窓情報を保存
        window_info = {
            'window_index': idx,
            'window_size': window_size,
            'stride': stride,
            'frame_start': frame_start,
            'frame_end': frame_end,
            'window_start_time': window_start_time,
            'window_end_time': window_end_time,
            'r_signal_in_window': array_to_full_string(r_signal_in_window),
            'g_signal_in_window': array_to_full_string(g_signal_in_window),
            'b_signal_in_window': array_to_full_string(b_signal_in_window),
            'raw_bvp_in_window': array_to_full_string(raw_bvp_signal_in_window),
            'filtered_bvp_in_window': array_to_full_string(filtered_bvp_signal_in_window),
            'ecg_bpm_mean': ecg_bpm_in_window_mean,
            'rppg_bpm': rppg_bpm,
            'original_bpm_MAE': original_mae,
            'bpm_MAE': bpm_MAE,
            'max_freq': fft_result_dic['max_freq'],
            'max_amplitude': fft_result_dic['max_amplitude'],
            'spectral_centroid': fft_result_dic['spectral_centroid'],
            'spectral_bandwidth': fft_result_dic['spectral_bandwidth'],
            'total_power': fft_result_dic['total_power'],
            'alpha_history': array_to_full_string(np.array(alpha_list))
        }
            
        all_results.append(window_info)

    # ============================================================================
    # 結果の集計と保存
    # ============================================================================

    # DataFrameに変換
    results_df = pd.DataFrame(all_results)

    # 結果を保存
    output_path = os.path.join(rootDir, SAVE_DIR, 'adaptive_removeDC_POS_high_mae_results.csv')
    results_df.to_csv(output_path, index=False)
    print(f"\n結果を保存しました: {output_path}")

    # 統計情報を表示
    print(f'\n統計情報 - {dataName} (適応的射影行列)')
    print('-' * 60)
    
    if len(all_results) > 0:
        results_df = pd.DataFrame(all_results)
        print(f'全データ数: {len(results_df)}')
        second_half_df = results_df[results_df['window_start_time'] >= SPLIT_TIME].copy()
        print(f'平均MAE: {second_half_df["bpm_MAE"].mean():.4f}')
        print(f'最小MAE: {second_half_df["bpm_MAE"].min():.4f}')
        print(f'最大MAE: {second_half_df["bpm_MAE"].max():.4f}')
    else:
        print('処理されたデータが存在しません')

In [ ]:
adaptive_results_path = os.path.join(rootDir, SAVE_DIR, 'adaptive_POS_high_mae_results2.csv')
print(f'適応的POS結果パス: {adaptive_results_path}')
adaptive_df = pd.read_csv(adaptive_results_path)

# 元のPOS結果を抽出
original_pos_results_path = os.path.join(rootDir, SAVE_DIR, 'window_analysis_' + dataName + '.csv')
print(f'元のPOS結果パス: {original_pos_results_path}')
original_pos_df = pd.read_csv(original_pos_results_path)
print(f'元のPOSデータ件数: {len(original_pos_df)}')

# 共通のwindow_indexを取得
common_indices = set(adaptive_df['window_index']).intersection(set(original_pos_df.index))
print(f'比較可能なウィンドウ数: {len(common_indices)}')
print(f'共通インデックス: {sorted(list(common_indices))[:10]}...')  # 最初の10個を表示

# 各ウィンドウの波形を比較
for window_idx in sorted(list(common_indices))[:-1]:  # 最初の5個を表示
    try:
        # 元のPOSデータ(window_indexはDataFrameのインデックス)
        original_row = original_pos_df.loc[window_idx]
        
        # 手法がPOSである行を選択(複数行ある場合)
        if isinstance(original_row, pd.DataFrame):
            original_row = original_row[original_row['bvp_method'] == 'cupy_POS'].iloc[0]

        # SPLIT_TIME以前の窓はスキップ
        if original_row['window_end_time'] <= SPLIT_TIME:
            continue
        
        original_bvp_str = original_row['filtered_bvp_in_window']
        original_bvp = np.fromstring(original_bvp_str[1:-1], sep=' ')
        print(f'\nウィンドウ {window_idx} の元のBVP信号長: {len(original_bvp)}')
        original_mae = original_row['bpm_MAE']
        original_bpm = original_row['rppg_bpm']
        
        # Red信号を取得
        r_signal_str = original_row['r_signal_in_window']
        r_signal = np.fromstring(r_signal_str[1:-1], sep=' ')
        print(f'ウィンドウ {window_idx} のRed信号長: {len(r_signal)}')
        
        # 時間情報を取得
        window_start_time = original_row['window_start_time']
        window_end_time = original_row['window_end_time']
        
        # 適応的POSデータ
        adaptive_row = adaptive_df[adaptive_df['window_index'] == window_idx].iloc[0]
        adaptive_bvp_str = adaptive_row['filtered_bvp_in_window']
        adaptive_bvp = np.fromstring(adaptive_bvp_str[1:-1], sep=' ')
        print(f'ウィンドウ {window_idx} の適応的BVP信号長: {len(adaptive_bvp)}')
        adaptive_mae = adaptive_row['bpm_MAE']
        adaptive_bpm = adaptive_row['rppg_bpm']
        
        # ECG真値
        ecg_bpm = original_row['ecg_bpm_mean']
        ecg_freq = ecg_bpm / 60.0  # BPMをHzに変換
        
        # Original POSの周波数解析
        original_fft_result = analyze_window_fft(original_bvp, fps)
        original_freqs = original_fft_result['frequencies']
        original_amplitudes = original_fft_result['amplitudes']
        original_peak_freq = original_fft_result['max_freq']
        
        # Adaptive POSの周波数解析
        adaptive_fft_result = analyze_window_fft(adaptive_bvp, fps)
        adaptive_freqs = adaptive_fft_result['frequencies']
        adaptive_amplitudes = adaptive_fft_result['amplitudes']
        adaptive_peak_freq = adaptive_fft_result['max_freq']
        
        # それぞれの時間軸を作成
        time_axis_r = np.arange(len(r_signal)) / fps
        time_axis_original = np.arange(len(original_bvp)) / fps
        time_axis_adaptive = np.arange(len(adaptive_bvp)) / fps
        
        # プロット (5行に変更: Red, Original BVP, Original FFT, Adaptive BVP, Adaptive FFT)
        fig, axes = plt.subplots(5, 1, figsize=(15, 18))
        
        # Red信号
        axes[0].plot(time_axis_r, r_signal, 'r-', linewidth=1.5, label='Red Signal')
        axes[0].set_title(f'Window {window_idx} - Red Signal\nTime: {window_start_time:.2f}s - {window_end_time:.2f}s', 
                         fontsize=12, fontweight='bold')
        axes[0].set_ylabel('Amplitude')
        axes[0].grid(True, alpha=0.3)
        axes[0].legend()
        
        # 元のPOS
        axes[1].plot(time_axis_original, original_bvp, 'b-', linewidth=1.5, label='Original POS')
        axes[1].set_title(f'Window {window_idx} - Original POS\nBPM: {original_bpm:.2f} (ECG: {ecg_bpm:.2f}), MAE: {original_mae:.2f}', 
                         fontsize=12, fontweight='bold')
        axes[1].set_ylabel('Amplitude')
        axes[1].grid(True, alpha=0.3)
        axes[1].legend()
        
        # 元のPOSの周波数解析
        axes[2].plot(original_freqs, original_amplitudes, 'b-', linewidth=1.5, label='Original POS FFT')
        axes[2].axvline(x=original_peak_freq, color='r', linestyle='--', linewidth=2, 
                       label=f'Peak: {original_peak_freq:.3f} Hz ({original_peak_freq*60:.1f} BPM)')
        axes[2].axvline(x=ecg_freq, color='g', linestyle='--', linewidth=2, 
                       label=f'ECG: {ecg_freq:.3f} Hz ({ecg_bpm:.1f} BPM)')
        axes[2].set_title(f'Window {window_idx} - Original POS Frequency Analysis', 
                         fontsize=12, fontweight='bold')
        axes[2].set_xlabel('Frequency [Hz]')
        axes[2].set_ylabel('Amplitude')
        axes[2].set_xlim([0.5, 3.0])  # 心拍数の範囲 (30-180 BPM)
        axes[2].grid(True, alpha=0.3)
        axes[2].legend()
        
        # 適応的POS
        axes[3].plot(time_axis_adaptive, adaptive_bvp, 'g-', linewidth=1.5, label='Adaptive POS')
        axes[3].set_title(f'Window {window_idx} - Adaptive POS\nBPM: {adaptive_bpm:.2f} (ECG: {ecg_bpm:.2f}), MAE: {adaptive_mae:.2f}', 
                         fontsize=12, fontweight='bold')
        axes[3].set_ylabel('Amplitude')
        axes[3].grid(True, alpha=0.3)
        axes[3].legend()
        
        # 適応的POSの周波数解析
        axes[4].plot(adaptive_freqs, adaptive_amplitudes, 'g-', linewidth=1.5, label='Adaptive POS FFT')
        axes[4].axvline(x=adaptive_peak_freq, color='r', linestyle='--', linewidth=2, 
                       label=f'Peak: {adaptive_peak_freq:.3f} Hz ({adaptive_peak_freq*60:.1f} BPM)')
        axes[4].axvline(x=ecg_freq, color='g', linestyle='--', linewidth=2, 
                       label=f'ECG: {ecg_freq:.3f} Hz ({ecg_bpm:.1f} BPM)')
        axes[4].set_title(f'Window {window_idx} - Adaptive POS Frequency Analysis', 
                         fontsize=12, fontweight='bold')
        axes[4].set_xlabel('Frequency [Hz]')
        axes[4].set_ylabel('Amplitude')
        axes[4].set_xlim([0.5, 3.0])  # 心拍数の範囲 (30-180 BPM)
        axes[4].grid(True, alpha=0.3)
        axes[4].legend()
        
        plt.tight_layout()
        
        # 保存
        save_path = os.path.join(rootDir, SAVE_DIR, f'comparison_window_{window_idx}.png')
        # plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f'保存: {save_path}')
        plt.show()
        plt.close()
        
    except Exception as e:
        print(f'ウィンドウ {window_idx} の処理中にエラー: {str(e)}')
        import traceback
        traceback.print_exc()
        continue

[-1,1,1]にした際に精度があがる理由を考察
- 1+i(t)がS2に残留するので、u_skinとu_pbvに直交するベクトルをかけて定数倍*(1+i(t))信号を抽出しプロット

In [ ]:
#v_pとv_sに直交するベクトルを求める
v_p = [0.33, 0.77, 0.53]
v_s = [0.37, 0.56, 0.75]

v_p = np.array(v_p)
v_s = np.array(v_s)
v_n = np.cross(v_p, v_s)
v_n = v_n / np.linalg.norm(v_n)
print("法線ベクトル v_n:", v_n)

# 検証
dot_pn = np.dot(v_p, v_n)
dot_sn = np.dot(v_s, v_n)
print("v_p・v_n:", dot_pn)
print("v_s・v_n:", dot_sn)

In [ ]:
import cupy as cp

# ============================================================================
# メイン処理部分（既存コードに組み込む）
# ============================================================================

# 結果を保存するリスト
all_results = []

for i in range(len(movie_paths)):
    print(f'Processing movie: {movie_paths[i]}')
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    window_analysis_path = os.path.join(rootDir, SAVE_DIR, f'window_analysis_{dataName}.csv')
    window_analysis_df = pd.read_csv(window_analysis_path)

    # cupy_POSかつMAE >= 10のデータを抽出
    high_mae_df = window_analysis_df[window_analysis_df['bvp_method'] == 'cupy_POS']
    high_mae_df = high_mae_df[high_mae_df['bpm_MAE'] >= 10]
    print(f'\n高MAEデータ (MAE >= 10) - {dataName}, 件数: {len(high_mae_df)}')

    if len(high_mae_df) == 0:
        print('  高MAEデータが存在しません')
        continue

    all_results = []
    
    # 各ウィンドウを処理
    for idx, row in high_mae_df.iterrows():
        window_index = row['window_index']
        window_size = row['window_size']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        original_mae = row['bpm_MAE']

        # window_sizeが4未満の場合はスキップ
        if window_size < 4:
            continue

        # window_startがSPLIT_TIME以前の場合はスキップ
        window_start_time = row['window_start_time']
        if window_start_time <= SPLIT_TIME:
            continue
        print(f"window length: {window_size}, processing window {window_index}: frames {frame_start}-{frame_end}, original MAE: {original_mae:.2f}")

        # RGB信号を抽出
        r_signal_in_window = np.fromstring(row['r_signal_in_window'][1:-1], sep=' ')
        g_signal_in_window = np.fromstring(row['g_signal_in_window'][1:-1], sep=' ')
        b_signal_in_window = np.fromstring(row['b_signal_in_window'][1:-1], sep=' ')

        rgb_signal = np.array([[r_signal_in_window, g_signal_in_window, b_signal_in_window]], dtype=np.float32)
        
        # ECG BPM(真値)
        ecg_bpm = row['ecg_bpm_in_window']
        ecg_bpm = np.fromstring(ecg_bpm[1:-1], sep=' ') if isinstance(ecg_bpm, str) else np.array([])
        ecg_bpm_in_window_mean = ecg_bpm.mean() if ecg_bpm is not None else None

        # CuPy配列に変換
        rgb_cupy = cp.asarray(rgb_signal)

        # POS法のパラメータ
        eps = 10**-9
        X = rgb_cupy
        fps_cupy = cp.float32(fps)
        e, c, f = X.shape  # e = #estimators, c = 3 rgb ch., f = #frames
        w = int(1.6 * fps_cupy)  # window length

        # 固定の拍動ベクトル(論文の式30)
        u_pbv = cp.array([0.33, 0.77, 0.53], dtype=cp.float32)

        # 肌色調ベクトル
        r_signal_in_window_square = r_signal_in_window ** 2
        g_signal_in_window_square = g_signal_in_window ** 2
        b_signal_in_window_square = b_signal_in_window ** 2

        normalized_r_signal_in_window = r_signal_in_window / np.sqrt(r_signal_in_window_square + g_signal_in_window_square + b_signal_in_window_square)
        normalized_g_signal_in_window = g_signal_in_window / np.sqrt(r_signal_in_window_square + g_signal_in_window_square + b_signal_in_window_square)
        normalized_b_signal_in_window = b_signal_in_window / np.sqrt(r_signal_in_window_square + g_signal_in_window_square + b_signal_in_window_square)

        skin_vector_array = np.array([normalized_r_signal_in_window, normalized_g_signal_in_window, normalized_b_signal_in_window])
        skin_vector = cp.mean(cp.asarray(skin_vector_array), axis=1)

        # POS法の標準化された肌色ベクトル(uskin)と血液量パルスベクトル(upbv)に直交する法線ベクトルを計算
        # uskin = [0.77, 0.51, 0.38] (CHROM論文より)
        u_skin = skin_vector / (cp.linalg.norm(skin_vector) + eps)
        # u_pbvとu_skinに直交するベクトルを求める
        uskin = cp.asnumpy(u_skin)
        upbv = cp.asnumpy(u_pbv)
        v_n = cp.cross(u_skin, u_pbv)
        v_n = v_n / cp.linalg.norm(v_n)
        v_n = cp.asnumpy(v_n)
        print(f"肌色調ベクトル uskin: {uskin}, 法線ベクトル v_n: {v_n}")

        # 検証
        dot_u_skin = np.dot(uskin, v_n)
        dot_upbv = np.dot(upbv, v_n)
        print("u_skin・v_n:", dot_u_skin)
        print("u_pbv・v_n:", dot_upbv)
        # uskinとupbvの両方に直交するベクトルを外積で計算
        normal_vector = cp.cross(uskin, u_pbv)
        # 正規化
        normal_vector = normal_vector / cp.linalg.norm(normal_vector)
        
        # 投影行列P(1x3の行ベクトル)
        P = cp.reshape(normal_vector, (1, 3))

        # 初期化
        intensity_signal = cp.zeros((e, f))

        # スライディングウィンドウループ
        for n in cp.arange(w, f):
            m = n - w + 1
            
            # 時間的正規化
            Cn = X[:, :, m:(n + 1)]
            M = 1.0 / (cp.mean(Cn, axis=2) + eps)
            M_expanded = cp.expand_dims(M, axis=2)
            Cn = cp.multiply(M_expanded, Cn)
            
            # 投影(uskinとupbvに直交する方向に投影してi(t)を抽出)
            # Cn shape: (e, 3, window_length)
            # P shape: (1, 3)
            # 投影結果 shape: (e, window_length)
            for estimator_idx in range(e):
                
                projected = cp.dot(P, Cn[estimator_idx, :, :]).flatten()
                # ゼロ平均化
                projected = projected - cp.mean(projected)
                # オーバーラップ加算
                intensity_signal[estimator_idx, m:(n + 1)] = cp.add(
                    intensity_signal[estimator_idx, m:(n + 1)], 
                    projected
                )
                # プロット用
                # plot_intensity_signal[estimator_idx, m:(n + 1)] = cp.dot(P, Cn[estimator_idx, :, :]).flatten()


                # plot_intensity_signalをプロット
                # fig, axes = plt.subplots(1, 1, figsize=(10, 4))
                # intensity_np = cp.asnumpy(plot_intensity_signal[0, m:(n + 1)])
                # axes.plot(intensity_np, color='red', label='Intensity Signal i(t)', linewidth=1.5)
                # axes.set_title(f'Window {window_index} - Frame {int(m)} to {int(n)} - Intensity Signal (orthogonal to uskin and upbv)')
                # axes.set_xlabel('Frame')
                # axes.set_ylabel('Amplitude')
                # axes.legend()
                # axes.grid(True, alpha=0.3)
                # plt.tight_layout()
                # plt.show()
                # plot_save_dir = os.path.join(rootDir, SAVE_DIR, 'normal_vector_diagnostics_plots')
                # os.makedirs(plot_save_dir, exist_ok=True)
                # plot_filename = f'window_{window_index}_{int(n)}_I1_signal.png'
                # plot_path = os.path.join(plot_save_dir, plot_filename)
                # plt.savefig(plot_path)
                # plt.close()
        # 最終的な強度変動信号をプロット
        fig, axes = plt.subplots(1, 1, figsize=(15, 4))
        intensity_np = cp.asnumpy(intensity_signal[0, :])
        axes.plot(intensity_np, color='red', label='Intensity Signal i(t) - Full', linewidth=1.5)
        axes.set_title(f'Window {window_index} - Full Intensity Signal (orthogonal to uskin and upbv)')
        axes.set_xlabel('Frame')
        axes.set_ylabel('Amplitude')
        axes.legend()
        axes.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        

        # NumPy配列に戻す
        bvp_cupy = intensity_signal
        bvp_numpy = cp.asnumpy(bvp_cupy)

        raw_bvp_signal = [bvp_numpy]
        bvp_signal = [bvp_numpy.copy()]

        # 後処理フィルタリング
        bvp_signal = vhr.BVP.apply_filter(
            bvp_signal,
            vhr.BVP.BPfilter,
            params={'order': 6, 'minHz': 0.5, 'maxHz': 2.0, 'fps': fps}
        )

        bvp_signal = vhr.BVP.apply_filter(bvp_signal, vhr.BVP.zeromean)

        raw_bvp_signal_in_window = raw_bvp_signal[0] if len(bvp_signal) > 0 else None
        filtered_bvp_signal_in_window = bvp_signal[0] if len(bvp_signal) > 0 else None

        # FFT解析
        raw_bvp_signal_in_window = raw_bvp_signal_in_window.flatten() if raw_bvp_signal_in_window is not None else None
        filtered_bvp_signal_in_window = filtered_bvp_signal_in_window.flatten()
        fft_result_dic = analyze_window_fft(filtered_bvp_signal_in_window, fps)

        # MAEの計算
        rppg_bpm = fft_result_dic['max_freq'] * 60
        rppg_freq = fft_result_dic['frequencies']
        rppg_amplitude = fft_result_dic['amplitudes']
        rppg_pwd = fft_result_dic['power_spectrum']

        bpm_MAE = np.abs(ecg_bpm_in_window_mean - rppg_bpm) if not np.isnan(ecg_bpm_in_window_mean) else np.nan

        # 結果をリストに追加
        # 窓情報を保存
        window_info = {
            'window_index': idx,
            'window_size': window_size,
            'stride': stride,
            'frame_start': frame_start,
            'frame_end': frame_end,
            'window_start_time': window_start_time,
            'window_end_time': window_end_time,
            'r_signal_in_window': array_to_full_string(r_signal_in_window),
            'g_signal_in_window': array_to_full_string(g_signal_in_window),
            'b_signal_in_window': array_to_full_string(b_signal_in_window),
            'raw_bvp_in_window': array_to_full_string(raw_bvp_signal_in_window),
            'filtered_bvp_in_window': array_to_full_string(filtered_bvp_signal_in_window),
            'ecg_bpm_mean': ecg_bpm_in_window_mean,
            'rppg_bpm': rppg_bpm,
            'original_bpm_MAE': original_mae,
            'bpm_MAE': bpm_MAE,
            'max_freq': fft_result_dic['max_freq'],
            'max_amplitude': fft_result_dic['max_amplitude'],
            'spectral_centroid': fft_result_dic['spectral_centroid'],
            'spectral_bandwidth': fft_result_dic['spectral_bandwidth'],
            'total_power': fft_result_dic['total_power'],
            'alpha_history': array_to_full_string(np.array(alpha_list))
        }
            
        all_results.append(window_info)

    # ============================================================================
    # 結果の集計と保存
    # ============================================================================

    # DataFrameに変換
    results_df = pd.DataFrame(all_results)

    # 結果を保存
    output_path = os.path.join(rootDir, SAVE_DIR, 'It_POS_high_mae_results.csv')
    # results_df.to_csv(output_path, index=False)
    print(f"\n結果を保存しました: {output_path}")

    # 統計情報を表示
    print(f'\n統計情報 - {dataName} (適応的射影行列)')
    print('-' * 60)
    
    if len(all_results) > 0:
        results_df = pd.DataFrame(all_results)
        print(f'全データ数: {len(results_df)}')
        second_half_df = results_df[results_df['window_start_time'] >= SPLIT_TIME].copy()
        print(f'平均MAE: {second_half_df["bpm_MAE"].mean():.4f}')
        print(f'最小MAE: {second_half_df["bpm_MAE"].min():.4f}')
        print(f'最大MAE: {second_half_df["bpm_MAE"].max():.4f}')
    else:
        print('処理されたデータが存在しません')